In [5]:
import numpy as np
import pandas as pd
import torch
from transformers import BertTokenizer, BertModel, BertForSequenceClassification
from sklearn.model_selection import train_test_split
import random

In [6]:
#import only 5% of dataset for performance enhancement
df = pd.read_csv('data/TweetsCOV19.csv', nrows=int(0.05 * pd.read_csv('data/TweetsCOV19.csv').shape[0]))

/tmp/ipykernel_41520/2828937271.py:2: DtypeWarning: Columns (0,1) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('data/TweetsCOV19.csv', nrows=int(0.05 * pd.read_csv('data/TweetsCOV19.csv').shape[0]))


In [7]:
#train test split
dftr = pd.DataFrame() #train
dfvl = pd.DataFrame() #validation
dfte = pd.DataFrame() #test
dfti = pd.DataFrame() #leon special

dftr["X"], dfti["X"], dftr["y"], dfti["y"] = train_test_split(df['TweetText'], df["Sentiment"], test_size=0.2, random_state=1)
dftr[["y_pos", "y_neg"]] = dftr["y"].str.split(' ', expand=True)
dftr[["y_pos", "y_neg"]] = dftr[["y_pos", "y_neg"]].astype(int)

dfvl["X"], dfte["X"], dfvl["y"], dfte["y"] = train_test_split(dfti["X"], dfti["y"], test_size=0.5, random_state=1)
dfte[["y_pos", "y_neg"]] = dfte["y"].str.split(' ', expand=True)
dfte[["y_pos", "y_neg"]] = dfte[["y_pos", "y_neg"]].astype(int)

dfvl[["y_pos", "y_neg"]] = dfvl["y"].str.split(' ', expand=True)
dfvl[["y_pos", "y_neg"]] = dfvl[["y_pos", "y_neg"]].astype(int)


In [8]:
from transformers import Trainer, TrainingArguments

In [9]:
from transformers import AutoModelForSequenceClassification
from transformers import TFAutoModelForSequenceClassification
from transformers import AutoTokenizer, AutoConfig

In [10]:
MODEL = f"cardiffnlp/twitter-roberta-base-sentiment-latest"
tokenizer = AutoTokenizer.from_pretrained(MODEL)
config = AutoConfig.from_pretrained(MODEL)
model = AutoModelForSequenceClassification.from_pretrained(MODEL)

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [11]:
dftr_list = dftr['X'].tolist()
dfvl_list = dftr['X'].tolist()
dfte_list = dftr['X'].tolist()

In [12]:
train_encodings = tokenizer(dftr_list, truncation=True, padding=True)
val_encodings = tokenizer(dfvl_list, truncation=True, padding=True)
test_encodings = tokenizer(dfte_list, truncation=True, padding=True)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [13]:
from transformers import Trainer, TrainingArguments
import accelerate
# import tensorrt

In [14]:
train_labels = dftr['y_pos'].tolist()
val_labels = dfvl['y_pos'].tolist()
test_labels = dfte['y_pos'].tolist()
#print(dfvl.columns)

In [15]:
train_int_labels = [int(label) for label in train_labels]
val_int_labels = [int(label) for label in val_labels]
test_int_labels = [int(label) for label in test_labels]


In [16]:
class TweetDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = TweetDataset(train_encodings, train_int_labels)
val_dataset = TweetDataset(val_encodings, val_int_labels)
test_dataset = TweetDataset(test_encodings, test_int_labels)

In [17]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

In [18]:
training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=3,              # total number of training epochs
    per_device_train_batch_size=8, #16,  # batch size per device during training
    per_device_eval_batch_size=32, #64,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=10,
    gradient_accumulation_steps = 2,
    learning_rate = 1e-5,
    fp16=True

)

#model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased")

trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=val_dataset             # evaluation dataset
)

trainer.train()

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


/home/leon/.local/share/virtualenvs/23fs_mlhc_p2-jtQQEGpx/lib/python3.10/site-packages/torch/cuda/__init__.py:546: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


ValueError: FP16 Mixed precision training with AMP or APEX (`--fp16`) and FP16 half precision evaluation (`--fp16_full_eval`) can only be used on CUDA devices.